In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [14]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.acceptance import f_decline, f_decline_R50, f_decline_R75, f_decline_R100, f_decline_mixed
from MaaSSim.acceptance import RA_kpi_veh, RA_kpi_pax

In [2]:
import h3
import folium
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import numpy as np
import seaborn as sns

In [38]:
params = MaaSSim.utils.get_config('glance.json')
params.vehicle_fixed_positions = True
params.times.patience = 600 #in second
params.dist_threshold = 20000 #in meter
params.dist_threshold_min = 3000
params.city = "Amsterdam, Netherlands"
params.paths.G = "../../data/graphs/Amsterdam.graphml"
params.paths.skim = "../../data/graphs/Amsterdam.csv"

params.d2d.fuel_cost = 0.25 #euro/km
params.platforms.comm_rate = 0
params.platforms.base_fare = 2
params.platforms.fare = 1.2 
params.platforms.min_fare = 2
params.speeds.ride=10
params.times.decline=20
params.assert_me = False

params.paths.requests = 'Amsterdam_requests_new.csv'

params.nP = 1000
params.nV = 100
params.simTime = 4

In [39]:
sim = MaaSSim.simulators.simulate(params=params)

29-06-22 19:06:47-WARNING-Setting up 4h simulation at 2022-06-29 17:06:52 for 100 vehicles and 1000 passengers in Amsterdam, Netherlands
29-06-22 19:06:50-INFO--------------------	Starting simulation	-------------------
29-06-22 19:06:51-INFO-pax  546  got to dest at                           2022-06-29 17:15:37
29-06-22 19:06:51-INFO-pax  359  got to dest at                           2022-06-29 17:17:02
29-06-22 19:06:51-INFO-pax  332  got to dest at                           2022-06-29 17:17:36
29-06-22 19:06:51-INFO-pax  219  got to dest at                           2022-06-29 17:17:46
29-06-22 19:06:51-INFO-pax   37  got to dest at                           2022-06-29 17:17:54
29-06-22 19:06:51-INFO-pax  495  got to dest at                           2022-06-29 17:18:15
29-06-22 19:06:51-INFO-pax  616  got to dest at                           2022-06-29 17:18:36
29-06-22 19:06:51-INFO-pax  338  got to dest at                           2022-06-29 17:18:37
29-06-22 19:06:51-INFO-pax  

In [7]:
def visualize_hexagons(hexagons, color="red", w=1, folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=w,color=color,fill= True)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [3]:
paths= "../../data/graphs/Amsterdam.graphml"
G = ox.load_graphml(paths)

In [119]:
# Visualiziation of hexagons on Amsterdam map
lat = []
lng = []
for i in G.nodes:
    lat.append(G.nodes[i]['y'])
    lng.append(G.nodes[i]['x'])
    
df = pd.DataFrame()
df['lat'] = lat
df['lng'] = lng

level = 7
hex_col = 'hex'+str(level)
df[hex_col] = df.apply(lambda x: h3.geo_to_h3(x.lat,x.lng,level),axis=1)
df = df.groupby(hex_col).size().to_frame('cnt').reset_index()

l = list(df[hex_col])
m = visualize_hexagons(l)
display(m)

In [11]:
df

,hex7,cnt
0,871968240ffffff,11
1,871968241ffffff,12
2,871968244ffffff,12
3,871968245ffffff,30
4,871968248ffffff,34
5,871968249ffffff,250
6,87196824cffffff,47
7,87196824dffffff,346
8,871968263ffffff,1
9,871968268ffffff,38


In [62]:
req = sim.requests
treq = sim.requests.loc[13].treq
treq

Timestamp('2022-06-29 17:08:53')

In [60]:
sim.t0 + pd.Timedelta(60, 's')

Timestamp('2022-06-29 17:07:52')

In [61]:
if sim.t0<treq<sim.t0 + pd.Timedelta(60, 's'):
    print('yes')

In [46]:
sim.requests

,pax_id,origin,destination,treq,tdep,ttrav,tarr,tdrop,shareable,schedule_id,dist,ride_id,position,sim_schedule
id,,,,,,,,,,,,,,
1,384,1429031808,46392809,2022-06-29 17:06:52,NaN,0 days 00:11:43,2022-06-29 17:18:35,NaN,False,NaN,7033,384,0,node time req_id od 0 ...
2,338,5002844220,46177782,2022-06-29 17:07:21,NaN,0 days 00:08:37,2022-06-29 17:15:58,NaN,False,NaN,5173,338,0,node time req_id od 0 ...
3,546,46344111,46353914,2022-06-29 17:07:24,NaN,0 days 00:06:25,2022-06-29 17:13:49,NaN,False,NaN,3851,546,0,node time req_id od 0 NaN...
4,495,29529317,46436272,2022-06-29 17:07:48,NaN,0 days 00:09:12,2022-06-29 17:17:00,NaN,False,NaN,5520,495,0,node time req_id od 0 NaN...
5,37,46414371,2772719495,2022-06-29 17:08:06,NaN,0 days 00:07:36,2022-06-29 17:15:42,NaN,False,NaN,4564,37,0,node time req_id od 0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,767,46286385,6443995600,2022-06-29 21:04:53,NaN,0 days 00:06:45,2022-06-29 21:11:38,NaN,False,NaN,4055,767,0,node time req_id od 0 ...
997,451,46302492,46431042,2022-06-29 21:05:33,NaN,0 days 00:06:42,2022-06-29 21:12:15,NaN,False,NaN,4025,451,0,node time req_id od 0 NaN...
998,680,1574292212,46357320,2022-06-29 21:05:35,NaN,0 days 00:06:54,2022-06-29 21:12:29,NaN,False,NaN,4149,680,0,node time req_id od 0 ...


In [136]:
req = sim.requests
req = req[sim.t0<=req.treq]
req = req[req.treq<sim.t0 + pd.Timedelta(60, 's')]

level = 7
df['nreq'] = 0 

for index, row  in req.iterrows():
    lat = G.nodes[row.origin]['y']
    lng = G.nodes[row.origin]['x']
    
    add = h3.geo_to_h3(lat,lng,level)
    
    index = df[df.hex7 == add].index
    df.at[index,'nreq'] = df.loc[index].nreq+1
    
df

,hex7,cnt,nreq
0,871968240ffffff,11,0
1,871968241ffffff,12,0
2,871968244ffffff,12,0
3,871968245ffffff,30,0
4,871968248ffffff,34,0
5,871968249ffffff,250,0
6,87196824cffffff,47,0
7,87196824dffffff,346,0
8,871968263ffffff,1,0
9,871968268ffffff,38,0
